## **MA5606 Tópicos Matemáticos en Aprendizaje de Máquinas, Redes Neuronales y Aprendizaje Profundo**

### **Tarea 1: Redes neuronales feedfoward y PINNs**

**Profesores: Claudio Muñoz y Joaquín Fontbona**

**Auxiliares: Javier Maass y Diego Olguín**

**Nombres integrantes: COMPLETAR**

**Instrucciones:**

- **Fecha de entrega:** **26 de abril de 2024, a las 23:59.**

- **Importante:** Si trabaja desde el link de Google Colab (muy recomendable para trabajar con DeepXDE) debe hacer un copia en su Drive antes de trabajar, de lo contrario se podrían no guardar sus códigos.

- Debe entregar un Jupyter Notebook (archivo .ipynb) con sus código en Python. Le pueden ser de mucha utilidad los códigos vistos en la actividad práctica.

- Sus códigos deben estar comentados y ordenados. Además, en formato texto debe colocar todas sus conclusiones y resultados pedidos que deban ser redactados.

- En todos los ejercicios se le pide hacer al menos un gráfico. Los gráficos que realicen deben ser claros, con títulos y nombres en los ejes, junto con leyendas si es que corresponde.

In [10]:
# Librerías

# Numpy y matplotlib, junto con seaborn, para gráficos un poco mejores
import numpy as np
import matplotlib.pyplot as plt

# Puede ser útil para hacer gráficos con barras de colores
from mpl_toolkits.axes_grid1 import make_axes_locatable

import seaborn as sns

sns.set_theme()

# PyTorch y módulos que serán necesario
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [11]:
# Clase para crear redes neuronales
class NeuralNetwork(nn.Module):
    """
    Clase que define una red neuronal con una cantidad de capas y neuronas por capa definida por el usuario.
    Es una subclase de nn.Module, la clase base para todos los módulos de PyTorch.
    """

    def __init__(
        self,
        dim_input: int,
        dim_output: int,
        n_hidden_layers: int,
        width: int,
        activation: callable,
    ) -> None:
        """
        Constructor de la clase, que recibe los parámetros necesarios para crear una red neuronal.

        Args:
            - dim_input (int): Dimensión de la entrada.
            - dim_output (int): Dimensión de la salida.
            - n_hidden_layers (int): Número de capas internas.
            - wide (int): Ancho de las capas internas.
            - activation (callable): Función de activación.

        Returns:
            - None

        Rasies:
            - AssertionError: Si dim_input, dim_output, n_hidden_layers o wide no son enteros positivos.
            - AssertionError: Si activation no es una función.
        """
        super().__init__()  # Inicia el constructor de la clase base (nn.Module)

        for int_input, name in zip(
            [dim_input, dim_output, n_hidden_layers, width],
            ["dim_input", "dim_output", "n_hidden_layers", "width"],
        ):
            assert (
                isinstance(int_input, int) and int_input > 0
            ), f"Error en los parámetros de la red. Error: {int_input} ({name}) debe ser un entero positivo."
        assert callable(
            activation
        ), "Error en los parámetros de la red. Error: activation debe ser una función."

        self.first_layer = nn.Linear(dim_input, width)
        self.inner_layers = nn.ModuleList(
            [nn.Linear(width, width) for _ in range(n_hidden_layers - 1)]
        )
        self.last_layer = nn.Linear(width, dim_output)

        self.activation = activation

    def forward(self, x) -> torch.Tensor:
        """
        Describe cómo se calcula la salida de la red a partir de la entrada.

        Args:
            - x (torch.Tensor): Tensor de entrada.

        Returns:
            - torch.Tensor: Tensor de salida.
        """
        x = self.activation(self.first_layer(x))
        for layer in self.inner_layers:
            x = self.activation(layer(x))
        x = self.last_layer(x)
        return x


"""
Notas:
- Un modulo de PyTorch es una red neuronal creada con PyTorch.
- nn.Module es la clase base para todos los módulos de PyTorch.
- nn.ModuleList es similar a las listas de python diseñada específicamente para almacenar modulos.
- nn.Linear(in_features(int), out_features(int)) crea una transformación afín desde una capa de in_features dimensiones a una capa de out_features dimensiones (y=xA^T + b).
- super().__init__() llama al constructor de la clase base. Es necesario para cualquier modulo de PyTorch.
- forward describe cómo se calcula la salida de la red a partir de la entrada.
"""

'\nNotas:\n- Un modulo de PyTorch es una red neuronal creada con PyTorch.\n- nn.Module es la clase base para todos los módulos de PyTorch.\n- nn.ModuleList es similar a las listas de python diseñada específicamente para almacenar modulos.\n- nn.Linear(in_features(int), out_features(int)) crea una transformación afín desde una capa de in_features dimensiones a una capa de out_features dimensiones (y=xA^T + b).\n- super().__init__() llama al constructor de la clase base. Es necesario para cualquier modulo de PyTorch.\n- forward describe cómo se calcula la salida de la red a partir de la entrada.\n'

## **Ejercicio 1**

El objetivo de este ejercicio es estudiar la aproximación vía redes neuronales del problema de frontera

$$ y''(x) + \frac{\pi^2}{4} y(x) = 0 $$
$$ y(-1) = y(1) = 0, \, y(0) = 1$$

Para ello utilice redes de 1 capa oculta de ancho $N$, usando $N = \{ 10, 20, 30, 100\}$. Entrene la red con 600 iteraciones del algoritmo ``Adam``.

En cada caso grafique la evolución de la función de pérdida en las iteraciones de entrenamiento, además de la red evaluada en puntos en el intervalo $[-1,1]$ y compare con la solución analítica:

$$ y(x) = \cos \left (\frac{\pi}{2} x \right ) $$

In [12]:
# Parametros
gamma = 0.01 # Learning rate.
N = [10, 20, 30, 100] # Anchos de las NN.
resolution = 100  # Cardinalidad de la partición de [-1,1].
iters = 600  # Iteraciones de entrenamiento.

# Listas de redes y optimizadores
neural_list = nn.ModuleList()
optimizer_list = []

# Creación de las redes y optimizadores
for n in N:
    NN = NeuralNetwork(
        dim_input=1, dim_output=1, n_hidden_layers=1, width=n, activation=F.tanh
    )
    neural_list.append(NN)

    optimizer = optim.Adam(NN.parameters(), lr=gamma)
    optimizer_list.append(optimizer)


# Iremos guardando la pérdida en cada iteración
loss_record = []

X = torch.rand(resolution, 1)


def calc_loss(NN):
    # Predicción de la red
    output = NN(X)

    # Calculo de la función de pérdida, en este caso es en media cuadrática
    loss = nn.MSELoss(output, y)

    return loss

## **Ejercicio 2**



Veremos nuevamente el problema de EDP de Helmholtz

$$ -\Delta u - k_0^2 u = f, \quad \Omega = (0,1)^2 $$
$$ u = 0, \quad ∂ Ω $$

Que tiene solución analítica

$$ u(x, y) =  \sin (k_0 x) \sin (k_0 y)$$

Cuando $f(x,y) = k_0^2 \sin (k_0 x) \sin (k_0 y)$. Considere $k_0 = 2 n \pi$ y entrene una red neuronal de 3 capas, 100 neuronas por capa, función de activación seno y 3000 iteraciones del optimizador Adam, que resuelva el problema para $n \in \{ 1, 3, 5 \}$. Grafique la función de pérdida en función de las iteraciones de entrenamiento y el resultado de la red, comparando este con la solución analítica. ¿Qué resultado observa para los distintos $n$ propuestos?

In [13]:
# Instalamos e importamos la librería
import deepxde as dde


# Importamos Tensorflow, que es el backend que utilizaremos
import tensorflow as tf

Using backend: pytorch
Other supported backends: tensorflow.compat.v1, tensorflow, jax, paddle.
paddle supports more examples now and is recommended.


In [14]:
# Solución!

### **Ejercicio 3**

Considere el problema de Poisson

$$ -\Delta u = x(1-x) + y(1-y), \quad \Omega = (0,1)^2 $$
$$ u = 0, \quad ∂ Ω $$

Que tiene solución analítica

$$ u(x, y) = \frac{1}{2} x(1-x)y(1-y)$$

Entrene una red neuronal que aproxime la solución al problema, para ello considere una red de 3 capas, 100 neuronas por cada, función de activatión tangente hiperbólica y 500 iteraciones del optimizador Adam. Grafique la función de pérdida en función de las iteraciones de entrenamiento y el resultado de la red, comparando este con la solución analítica del problema.


In [15]:
# Solución!